<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/045_TaggingTools__forOrganization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧠 Tagging Tools for Organization

### 🧩 What’s Happening?

You’re using a **decorator** (`@register_tool`) to:

* **Register functions as tools** (e.g. `read_file`, `write_file`)
* **Attach metadata via tags** like `"file_operations"`, `"read"`, `"database"`

This decorator:

* Adds the tool to a global `tools` registry (by name)
* Adds the tool to `tools_by_tag`, so you can **query by functionality**

---

### ✅ Why Is This Powerful?

| Feature                    | Benefit                                                                        |
| -------------------------- | ------------------------------------------------------------------------------ |
| 🔍 Tags group tools        | You can organize tools by role or capability (e.g., `"write"` or `"database"`) |
| 🧱 Build registries by tag | Quickly construct focused agents with just the tools they need                 |
| 📦 Clean architecture      | Tools stay decoupled from agents but are easy to retrieve                      |
| 🔐 Safer agent design      | Only include tools appropriate for the agent's purpose                         |
| 🔄 Reusability             | Multiple agents can reuse the same tool under different tags                   |




## 🧠 **📦 Data Analysis Agent**

Build a Data Analysis Agent that can:

* Load a dataset
* Perform basic statistics
* Generate visualizations

We'll organize these tools using tags like:

| Tag               | Purpose                |
| ----------------- | ---------------------- |
| `"data"`          | Data loading / parsing |
| `"statistics"`    | Mean, median, etc.     |
| `"visualization"` | Generate charts        |

---

## ✅ Step 1: Set Up Global Registries and Decorator

```python
# Global tool registries
tools = {}
tools_by_tag = {}

def register_tool(tags=None, parameters_override=None):
    def decorator(func):
        tool_name = func.__name__
        tools[tool_name] = func

        # Register under each tag
        if tags:
            for tag in tags:
                tools_by_tag.setdefault(tag, []).append(tool_name)

        # Store parameters override (optional)
        func.parameters_override = parameters_override or {}
        func.tool_tags = tags or []
        return func
    return decorator
```

---

## ✅ Step 2: Define the Tools with Tags

```python
from typing import List, Dict
import statistics
import matplotlib.pyplot as plt

@register_tool(tags=["data"])
def load_csv(file_path: str) -> List[float]:
    """Mock loading of CSV: returns a list of numbers."""
    return [10, 20, 30, 40, 50]

@register_tool(tags=["statistics"])
def compute_statistics(data: List[float]) -> Dict:
    """Returns basic statistics."""
    return {
        "mean": statistics.mean(data),
        "median": statistics.median(data),
        "stdev": statistics.stdev(data)
    }

@register_tool(tags=["visualization"])
def generate_plot(data: List[float]) -> str:
    """Generates a basic line plot and returns image path."""
    plt.plot(data)
    path = "plot.png"
    plt.savefig(path)
    plt.close()
    return path
```

---

## ✅ Step 3: Check What’s Registered

```python
print("All tools:", list(tools.keys()))
print("Visualization tools:", tools_by_tag["visualization"])
print("Statistics tools:", tools_by_tag["statistics"])
```



A **decorator** in Python is a special kind of function — not a naming convention — that lets you **modify or enhance the behavior of other functions or classes**, *without changing their actual code*.

---

## 🧠 Simple Definition:

A **decorator** is a function that **wraps another function**, often to add extra behavior (like logging, registering, validation, etc.).

---

## 🔧 Anatomy of a Decorator

Here’s the core idea:

```python
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Before the function runs")
        result = func(*args, **kwargs)
        print("After the function runs")
        return result
    return wrapper
```

You use it like this:

```python
@my_decorator
def say_hello():
    print("Hello!")

say_hello()
```

**Output:**

```
Before the function runs
Hello!
After the function runs
```

The `@my_decorator` syntax is shorthand for:

```python
say_hello = my_decorator(say_hello)
```

---

## 🎯 In Our Case (Tool Registration)

We're using a decorator to **tag and register tools** automatically:

```python
@register_tool(tags=["data"])
def load_csv(file_path: str):
    ...
```

This means:

* You don’t have to manually add this function to a registry.
* It stores metadata (tags, parameters) **right alongside the function**, which is clean and powerful.



## ✅ Why Decorators Are Useful

Here’s a slightly deeper breakdown of how decorators shine in different areas:

| Use Case             | What It Does                                                             |
| -------------------- | ------------------------------------------------------------------------ |
| **Logging**          | Tracks function calls and arguments, great for debugging.                |
| **Caching**          | Stores results so expensive functions don't rerun unnecessarily.         |
| **Access Control**   | Checks if user/session/token is valid before continuing.                 |
| **Registration**     | Adds the function to a global registry — **perfect for agents + tools**. |
| **Validation**       | Ensures inputs conform to a schema, or outputs match expectations.       |
| **Timing/Profiling** | Measures how long a function takes to run.                               |

---

## 🧠 Why the Name “Decorator”?

Think of it like this:

> A **decorator** "wraps" or "decorates" an existing function to **add features** — like putting sprinkles on a cupcake without baking a whole new one.

It's **not changing the core logic**, just enhancing it from the outside.


---

## 🛠️ Think of Decorators As:

* ✨ *Dynamic plug-ins*
* 🧩 *Middleware*
* 🔌 *Hooks*

All built into Python’s syntax with just `@`.


This *is* a different approach than the `ActionRegistry` you were using before, and here's **why**:

---

## 🔁 Comparing Two Patterns

| Feature           | `ActionRegistry`                      | `register_tool` + `tools` dictionary   |
| ----------------- | ------------------------------------- | -------------------------------------- |
| **Structure**     | Explicit class-based registry         | Decorator-based with global dicts      |
| **Registration**  | Manual: you call `.register(...)`     | Automatic: happens when function loads |
| **Use Case**      | GAME framework agents                 | Lightweight, auto-organized tools      |
| **Best For**      | Full agent loop, structured execution | Dynamic tool discovery, quick tools    |
| **Extensibility** | High control over schema, logic       | High automation and ease of use        |

---

## 💡 So Why Use `register_tool` + Dicts Now?

Because when building **lots of tools**, especially in large teams or evolving projects:

* ✅ You don’t want to **manually register** each tool.
* ✅ You want to **group tools by purpose** using `tags`.
* ✅ You want **auto-discovery** so agents can be composed flexibly.

The decorator-based approach makes this *automatic* and *centralized*.

---

## 📦 Under the Hood

Behind the scenes, the decorator does the same thing as your `ActionRegistry`, but:

* It adds metadata (e.g., name, tags, parameters) to a global registry.
* It simplifies tool management — you don’t have to create `Action(...)` instances manually.
* It makes it easy to filter tools by purpose using `tags`.

You could think of it like this:

```python
# ActionRegistry
manual_registry = {
    "tool_name": Action(...)
}

# register_tool
auto_registry = {
    "tool_name": {
        "function": ...,
        "parameters": ...,
        "tags": ...
    }
}
```

They both store tools — just in **different formats for different needs**.

---

## ✨ Bonus: You Can Use Both Together

In a GAME agent, you could build an `ActionRegistry` dynamically *from* the `tools` dictionary collected via decorators. Best of both worlds!


